In [84]:
import pandas as pd
import numpy as np
from scipy import stats

In [85]:
gnns_df = pd.read_csv('../reports/results/sns-friendly/gnns.csv')
baseline_df = pd.read_csv('../reports/results/sns-friendly/baseline.csv')

In [86]:
gnns_df.head()

,conv,dataset,flow,test_acc,Network
0,GCNConv,cora,SYM,0.782670,original
1,GCNConv,cora,SYM,0.768150,original
2,GCNConv,cora,SYM,0.788759,original
3,GCNConv,cora,SYM,0.776581,original
4,GCNConv,cora,SYM,0.753162,original


In [87]:
for entry in gnns_df[gnns_df.dataset=='cora_full'].groupby(['dataset', 'conv', 'flow', 'Network']).groups:
    print(entry)

('cora_full', 'GATConv', 'SYM', 'conf')
('cora_full', 'GATConv', 'SYM', 'original')
('cora_full', 'GATConv', 'SYM', 'random')
('cora_full', 'GATConv', 'SYM', 'sbm')
('cora_full', 'GCNConv', 'SYM', 'conf')
('cora_full', 'GCNConv', 'SYM', 'original')
('cora_full', 'GCNConv', 'SYM', 'random')
('cora_full', 'GCNConv', 'SYM', 'sbm')
('cora_full', 'SAGEConv', 'SYM', 'conf')
('cora_full', 'SAGEConv', 'SYM', 'original')
('cora_full', 'SAGEConv', 'SYM', 'random')
('cora_full', 'SAGEConv', 'SYM', 'sbm')


In [88]:
gnns_df.groupby(['dataset', 'conv', 'flow', 'Network']).ngroups

92

In [90]:
arr1 = gnns_df[(gnns_df.dataset=='cora')&(gnns_df.conv=='GATConv')&(gnns_df.flow=='SYM')&(gnns_df.Network=='original')].test_acc.values

In [91]:
arr2 = baseline_df[baseline_df.dataset=='cora'].test_acc.values

In [92]:
_, p_value = stats.ttest_ind(arr1, arr2, equal_var=False)

In [93]:
p_value

4.521912231115878e-107

In [94]:
baseline_df.head()

,dataset,method,test_acc
0,cora,baseline,0.596253
1,cora,baseline,0.589696
2,cora,baseline,0.608431
3,cora,baseline,0.574707
4,cora,baseline,0.591101


In [102]:
statistical_significances = []
for entry in gnns_df.groupby(['dataset', 'conv', 'flow', 'Network']).groups:
    baseline_values = baseline_df[baseline_df.dataset==entry[0]].test_acc.values
    approach_values = gnns_df[(gnns_df.dataset==entry[0])&(gnns_df.conv==entry[1])&(gnns_df.flow==entry[2])&(gnns_df.Network==entry[3])].test_acc.values
    _, p_value = stats.ttest_ind(baseline_values, approach_values, equal_var=False)
    baseline_mean = np.mean(baseline_values)
    approach_mean = np.mean(approach_values)
    num_significance_tests = len(gnns_df[gnns_df.dataset==entry[0]].groupby(['dataset', 'conv', 'flow', 'Network']))
    alpha = 0.001
    _, p_value_normal_baseline = stats.normaltest(baseline_values)
    _, p_value_normal_approach = stats.normaltest(approach_values)
    statistical_significances.append({
        'dataset':entry[0],
        'conv':entry[1],
        'flow':entry[2],
        'Network':entry[3],
        'Baseline normal':p_value_normal_baseline>alpha,
        'Approach normal:':p_value_normal_approach>alpha,
        'p-value':p_value,
        'p-value<0.05':p_value<0.05,
        'p-value<0.05 (Bonferroni)':p_value<(0.05/num_significance_tests),
        'p-value<0.01':p_value<0.01,
        'p-value<0.01 (Bonferroni)':p_value<(0.01/num_significance_tests),
        'Baseline mean':baseline_mean,
        'Approach mean':approach_mean,
        'Beats baseline':approach_mean>baseline_mean
    })

In [104]:
statistical_significances_df = pd.DataFrame(statistical_significances)

In [105]:
statistical_significances_df[(statistical_significances_df.Network=='original')]

,dataset,conv,flow,Network,Baseline normal,Approach normal:,p-value,p-value<0.05,p-value<0.05 (Bonferroni),p-value<0.01,p-value<0.01 (Bonferroni),Baseline mean,Approach mean,Beats baseline
1,citeseer,GATConv,SYM,original,True,False,1.728788e-81,True,True,True,True,0.620215,0.733422,True
5,citeseer,GCNConv,SYM,original,True,False,1.703898e-76,True,True,True,True,0.620215,0.721310,True
9,citeseer,RGCNConv,SYM,original,True,False,1.113763e-30,True,True,True,True,0.620215,0.652624,True
13,citeseer,SAGEConv,SYM,original,True,False,1.729018e-77,True,True,True,True,0.620215,0.722808,True
17,cora,GATConv,SYM,original,True,False,4.521912e-107,True,True,True,True,0.579129,0.803943,True
21,cora,GCNConv,SYM,original,True,False,8.898874e-107,True,True,True,True,0.579129,0.812407,True
25,cora,RGCNConv,SYM,original,True,False,1.929011e-100,True,True,True,True,0.579129,0.774159,True
29,cora,SAGEConv,SYM,original,True,False,4.651399e-107,True,True,True,True,0.579129,0.811377,True
33,cora_full,GATConv,SYM,original,True,False,1.351582e-187,True,True,True,True,0.408393,0.549928,True
37,cora_full,GCNConv,SYM,original,True,False,4.710571e-151,True,True,True,True,0.408393,0.547800,True


In [103]:
statistical_significances_df[(statistical_significances_df.Network=='conf')]

,dataset,conv,flow,Network,Baseline means normally distributed,Approach means normally distributed:,p-value,p-value<0.05,p-value<0.05 (Bonferroni corrected),p-value<0.01,p-value<0.01 (Bonferroni corrected)
1,citeseer,GATConv,SYM,original,True,False,1.728788e-81,True,True,True,True
5,citeseer,GCNConv,SYM,original,True,False,1.703898e-76,True,True,True,True
9,citeseer,RGCNConv,SYM,original,True,False,1.113763e-30,True,True,True,True
13,citeseer,SAGEConv,SYM,original,True,False,1.729018e-77,True,True,True,True
17,cora,GATConv,SYM,original,True,False,4.521912e-107,True,True,True,True
21,cora,GCNConv,SYM,original,True,False,8.898874e-107,True,True,True,True
25,cora,RGCNConv,SYM,original,True,False,1.929011e-100,True,True,True,True
29,cora,SAGEConv,SYM,original,True,False,4.651399e-107,True,True,True,True
33,cora_full,GATConv,SYM,original,True,False,1.351582e-187,True,True,True,True
37,cora_full,GCNConv,SYM,original,True,False,4.710571e-151,True,True,True,True
